In [ ]:
# Imports
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import random
import time as t
from config import email
from config import password

In [ ]:
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

In [ ]:
username =
password =
#login to saltybet hit enter and wait 5 seconds for page to load
url = 'https://www.saltybet.com/authenticate?signin=1'
browser.visit(url)
#browser.find_by_id('email').click()
browser.find_by_id("email").fill(f'{username}')
browser.find_by_id("pword").fill(f'{password}\n')
# ActionChains(browser).move_to_element(text_input).send_keys_to_element(text_input, "mdicky93@gmail.com").perform()
t.sleep(5)

In [ ]:
def bet_loop():
    print('loop start')
    #check to make sure the bet was accepted before moving on
    betconfirm = browser.is_element_present_by_id('betconfirm')
    if betconfirm == True:
        #don't want to constantly keep clicking random until match starts. bet confirm appears once you bet and disappears upon match start.
        i = 0
        while betconfirm == True:
            if i == 0:
                print('bet confirmed')
            betconfirm = browser.is_element_present_by_id('betconfirm')
            t.sleep(5)
            i += 1
        #looking for the arrow in the odds section after match starts as an idicator that match is ongoing.
        i = 0
        while browser.find_by_id('betstatus').value == 'Bets are OPEN!' or browser.find_by_id('betstatus').value == 'Bets are locked until the next match.':
            t.sleep(0.2)
            if i == 0:
                print('in a match')
            i += 1
        while browser.find_by_id('betstatus').value != 'Bets are OPEN!':
            t.sleep(1)
    else:
        print('bet not confirmed')
        t.sleep(2)
    print('loop complete')

In [ ]:
def bet_who(bet_amount):
    player1 = browser.find_by_id('player1')
    player2 = browser.find_by_id('player2')
    try:
        #had to use a try except here as clicking when nothing is there causes errors. I'm sure this can be written better. this finds and clicks all in button
        allin=browser.find_by_id(bet_amount)
        allin.click()
        t.sleep(0.2)
        #random click of red or blue.
        coinflip=random.choice([0,1])
        if coinflip == 1:
            player2.click()
            print('bet on blue')
            t.sleep(0.2)
        else:
            player1.click()
            print('bet on red')
            t.sleep(0.2)
        print('go to loop')
        bet_loop()
    except:
        t.sleep(1)

In [ ]:
u_input = int(input('How many matches would you like to run?'))
random.seed()
for i in range(u_input):
    #t_time = browser.is_element_present_by_id('tournament-note')
    #used is_element_not instead of is_element_present as it runs almost 2 seconds faster, don't know why.
    t_time = browser.is_element_not_present_by_css('span[class="dollar purpletext"][id="balance"]')
    #if the purple text of balance is present we begin betting
    while t_time != False:
        t.sleep(30)
    while browser.find_by_id('betstatus').value != 'Bets are OPEN!':
        t.sleep(5)
    if t_time == False and int(browser.find_by_id('balance').value.replace(',',"")) <=4000:
        print(f'{i} tournament all in')
        print(browser.find_by_id('balance').value)
        bet_who('interval10')
    elif t_time == False and int(browser.find_by_id('balance').value.replace(',',"")) <=7000:
        print(f'{i} tournament 50 percent')
        print(browser.find_by_id('balance').value)
        bet_who('interval5')
    elif t_time == False and int(browser.find_by_id('balance').value.replace(',',"")) > 7000:
        print(f'{i} tournament 10 percent')
        print(browser.find_by_id('balance').value)
        bet_who('interval1')
browser.quit()